In [22]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

## 1. Crawl single site

In [23]:
def get_title(url): 
    res = requests.get(url) #get the website, return request.Response object
    #print(res.status_code) #statu_code: return 200(found web), 404(not found)
    soup = BeautifulSoup(res.text, 'html.parser')
    us_news_div = soup.find_all('div', re.compile('article-content'))
    
    title_list = []
    for i in range(len(us_news_div)):
        us_news_h3 = us_news_div[i].find_all('h3', recursive=False) #header
        us_news_a = us_news_h3[0].find_all('a', recursive=False) #anchor tag
        for index, item in enumerate(us_news_a[:]):
            title = item.text.strip()
            title_list.append(title)
    return title_list

In [28]:
def get_url(title_list):
    title_name = ''
    url_list = []
    pattern = re.compile('([^\s\w]|_)+')

    for i in range(len(title_list)):
        title = re.sub("[‘’]", '', title_list[i])
        #print(title)
        strippedList = pattern.sub(' ', title)
        a = strippedList.split(" ")
        empty_string = ''
        if empty_string in a:
            a = [x for x in a if x != '']
            title_name = '-'.join(a)
        else:
            title_name = '-'.join(a)
        url = 'https://www.infowars.com/'+title_name
        url_list.append(url)        
    return url_list

In [29]:
def get_content(url):
    res_content = requests.get(url) #get the website, return request.Response object
    if(res_content.status_code == 200):
        soup2 = BeautifulSoup(res_content.text, 'html.parser')
        #article = soup2.find_all('article')
        #content = article[0].find_all('p')
        
        full_content = ""
        
        for p in soup2.select('article > p'):
            if p.div or p.a or p.script:  # if div is present - skip
                continue
            full_content = full_content+p.text.strip()
        return full_content
    else:
        return

In [30]:
url1 = 'https://www.infowars.com/news/'

In [31]:
content_list = []
title_list = get_title(url1)
url_list = get_url(title_list)
for i in range(len(title_list)):
    content_list.append(get_content(url_list[i]))

In [32]:
#Dataframe: News | URL | Content
data = {'Title':title_list,'URL':url_list,'Content':content_list}
df = pd.DataFrame(data)
df = df[['Title', 'URL', 'Content']]
df.head()

,Title,URL,Content
0,MS-13 Member Gets 30 Years for Hacking a Man w...,https://www.infowars.com/MS-13-Member-Gets-30-...,A Virginia judge sentenced an MS-13 member to ...
1,Get On Alternative Social Media,https://www.infowars.com/Get-On-Alternative-So...,Big Tech is committed to the leftist agenda an...
2,Truth about CIA’s Illegal MKUltra Mind-control...,https://www.infowars.com/Truth-about-CIAs-Ille...,DISTURBING details of secret mind-control expe...
3,CNN Blames Trump For Bomb Threat: POTUS ‘Inspi...,https://www.infowars.com/CNN-Blames-Trump-For-...,“We were told to evacuate the building and to ...
4,Former Dem Comms Director Arrested For Child Porn,https://www.infowars.com/Former-Dem-Comms-Dire...,The former communications director for the Ark...


In [33]:
df.to_csv('result.csv')

# 2. Crawl Multiple Web Pages
## Dataframe: News | URL | Content

In [34]:
url = 'https://www.infowars.com'
res = requests.get(url) #get the website, return request.Response object
print(res.status_code) #statu_code: return 200(found web), 404(not found)

200


In [35]:
#print(res.text)

In [36]:
def get_tag(res, tag):   
    soup = BeautifulSoup(res.text, 'html.parser')
    tag_li = soup.find_all('li', re.compile(tag))
    pattern = re.compile('([^\s\w]|_)+')
    tag_list = []
    
    for i in range(len(tag_li)):
        tag_a = tag_li[i].find_all('a', recursive=False)
        for index, item in enumerate(tag_a[:]):
            tag = item.text.strip()
            tag = pattern.sub(' ', re.sub("[‘’.]", '', tag))
            tag_list.append(tag)

    return list(set(tag_list))

In [37]:
def get_category_url(title_list):
    title_name = ''
    url_list = []
    pattern = re.compile('([^\s\w]|_)+')

    for i in range(len(title_list)):
        title = re.sub("[‘’]", '', title_list[i])
        #print(title)
        strippedList = pattern.sub(' ', title)
        a = strippedList.split(" ")
        empty_string = ''
        if empty_string in a:
            a = [x for x in a if x != '']
            title_name = '-'.join(a)
        else:
            title_name = '-'.join(a)
        url = 'https://www.infowars.com/category/'+title_name
        url_list.append(url)        
    return url_list

In [38]:
news_tag = 'menu-item menu-item-type-taxonomy menu-item-object-category'
category_url_list = get_category_url(get_tag(res, news_tag))
category_url_list

['https://www.infowars.com/category/Government',
 'https://www.infowars.com/category/US-News',
 'https://www.infowars.com/category/World-News',
 'https://www.infowars.com/category/Science-Technology',
 'https://www.infowars.com/category/Hot-News',
 'https://www.infowars.com/category/Globalism',
 'https://www.infowars.com/category/Health',
 'https://www.infowars.com/category/World-at-War',
 'https://www.infowars.com/category/Economy',
 'https://www.infowars.com/category/Special-Reports']

In [39]:
#make sure the news webpages exist
for i in range(len(category_url_list)):
    res = requests.get(category_url_list[i]) #get the website, return request.Response object
    print(res.status_code) #statu_code: return 200(found web), 404(not found)

200
200
200
200
200
200
200
200
200
200


In [40]:
category_url_list

['https://www.infowars.com/category/Government',
 'https://www.infowars.com/category/US-News',
 'https://www.infowars.com/category/World-News',
 'https://www.infowars.com/category/Science-Technology',
 'https://www.infowars.com/category/Hot-News',
 'https://www.infowars.com/category/Globalism',
 'https://www.infowars.com/category/Health',
 'https://www.infowars.com/category/World-at-War',
 'https://www.infowars.com/category/Economy',
 'https://www.infowars.com/category/Special-Reports']

In [41]:
get_title(category_url_list[0])

['Trump Picks William Barr as Next Attorney General',
 'Sen. Ted Cruz, Others Announce Bill to Fund $25 Billion Border Wall',
 'What the Fake History of Guns Can Teach Us',
 'Clinton Foundation Whistleblowers Produce Devastating Evidence',
 'Dem Shopping List for Gun Control',
 'House Passes Bill to Delay Gov Shutdown, Border Wall Funding',
 'Mexico’s New Leader Eager to Talk to Trump About Immigration',
 'Kamala Harris Aide Resigns After Harassment, Retaliation Settlement Surfaces',
 'House Democrats Plan Push to Criminalize Private Gun Sales',
 'Trump Task Force Lets Postal Service Hike Prices For Amazon',
 'Sen. Grassley Wants Answers About FBI Raid on Clinton Foundation Whistleblower',
 'Roger Stone: Trump Doesn’t Know He Signed Free Speech Killing Agreement',
 'Christmas-Themed Satanic Sculpture Installed At Illinois Statehouse',
 'Roger Stone Calls Out Adam Schiff; If You Have Evidence, Present It Now',
 'Watch: D’Souza DESTROYS “Proud Democrat” in Heated Q&A',
 'Calif. Dems Plan

In [42]:
news_title_list = []
for i in range(len(category_url_list)):
    title_list = get_title(category_url_list[i])
    news_title_list = news_title_list + title_list
    news_title_list = list(set(news_title_list))

In [43]:
len(news_title_list)

319

In [ ]:
new_content_list = []
multi_page_url_list = get_url(news_title_list)
for i in range(len(news_title_list)):
    new_content_list.append(get_content(multi_page_url_list[i]))

In [ ]:
#Dataframe: News | URL | Content
data = {'Title':news_title_list,'URL':multi_page_url_list,'Content':new_content_list}
df = pd.DataFrame(data)
df = df[['Title', 'URL', 'Content']]
df.head()

In [ ]:
df.to_csv('result.csv')

## 3. Crawl Breaking News 

In [45]:
url_breaking = "https://www.infowars.com/breaking-news/"
#https://www.infowars.com/breaking-news/page/2/

In [46]:
def get_article_title(url, num_of_pages): 
    page = 1
    full_title_list = []
    while page <= num_of_pages:
        url_2 = url + "page/%s/" % page #get multiple page with "load page button"
        res = requests.get(url_2)
        soup = BeautifulSoup(res.text, 'html.parser')
        us_news_div = soup.find_all('article', re.compile('post-'))
        
        title_list = []
        for i in range(len(us_news_div)):
            us_news_a = us_news_div[i].find_all('a')
            us_news_h3 = us_news_a[0].find_all('h3')

            for index, item in enumerate(us_news_h3[:]):
                title = item.text.strip()
                #print(title)
                title_list.append(title)
        
        full_title_list.extend(title_list)
        page += 1

    return full_title_list


In [70]:
content_list = []
title_list = get_article_title(url_breaking, 500)
url_list = get_url(title_list)
for i in range(len(title_list)):
    content_list.append(get_content(url_list[i]))

In [71]:
data = {'Title':title_list,'URL':url_list,'Content':content_list}
df = pd.DataFrame(data)
df = df[['Title', 'URL', 'Content']]
# df

In [ ]:
df.to_csv('infowars5000_new.csv')